# First Project: Impact of Weather on Crime in Austin

In [ ]:
# Dependencies and Setup 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key

# Output File (CSV)

## Perform API Call - Austin Crime Data (data.austintexas.gov)

## Perform API Call - Weather Data (Dark Sky)

## Merge Data Sources (Crime and Weather Data)

## Clean Data 

## Plotting Part 1 - Crime Type vs. Weather Variables 

#### Crime Type vs. Temperature

#### Crime Type vs. Atmospheric Pressure

#### Crime Type vs. Humidity

#### Crime Type vs. Moon Phase

## Plotting Part 2 - Crime Location Type vs. Weather Variables 

#### Crime Location Type vs. Temperature

#### Crime Location Type vs. Atmospheric Pressure

#### Crime Location Type vs. Humidity

#### Crime Location Type vs. Moon Phase 

## Written Analysis and Conclusions